# Chapter 8 - Generalization and Function Approximation

This chapter explores methods to use RL beyond the simple environments presented so far, in situations where the state space, the action space or both are larger (or infinite) and cannot be represented effectively by the tabular framework.

### Exercise 8.1

Show that table-lookup TD($\lambda$) is a special case of general TD($\lambda$) as given by equations (8.5 - 8.7).

__Answer__: Since we are in the tabular case, there is a finite number of states, $\lvert \mathcal{S} \rvert$. So we can index the states arbitrarily by an integer like $s_1$, $s_2$, ..., $s_{\lvert \mathcal{S} \rvert}$ (there is no temporal meaning associated with this index, it's just a way to uniquely identify the states of the MDP). We can now represent the table-lookup values by using a simple vector $\theta \in \mathbb{R}^{\lvert \mathcal{S} \rvert}$ and define the value of any state $s_k \in \mathcal{S}$ as $V(s_k) \doteq \theta(k)$, the $k$-th component of the vector $\theta$. If we refer to the value of a state at a certain time step $t$, we can also write $V_t(s_k) \doteq \theta_t(k)$, to indicate that the vector $\theta$ itself is updated.

The $\delta_t$ remains the same as in equation (8.6) and the eligibility trace $\vec{e}$ is now a vector of dimension ${\lvert \mathcal{S} \rvert}$, containing the traces for all states $s \in \mathcal{S}$ at once. From the definition of $V$, the gradient of $V_t(s_k)$ w.r.t $\theta$ is a one-hot vector of zeros on all positions except in position $k$ where it is $1$.

Therefore, the update of $V$ after each step is replaced with identical results by simply updating the vector $\theta$ according to equation (8.5), for all steps at once. So the general TD($\lambda$) can easily represent a tabular TD($\lambda$) with these simple conventions.

### Exercise 8.2

_State aggregation_ is a simple form of generalizing function approximation in which states are grouped together, with one table entry (value estimate) used for each group. Whenever a state in a group is encountered, the group's entry is used to determine the state's value, and when the state is updated, the group's entry is updated. Show that this kind of aggregation is a special kind of gradient method such as (8.3).

__Answer__: Let's denote as usually the set of states $\mathcal{S}$ (not necessarily finite), the set of groups $\mathcal{G}$ (must be finite) and a function $g : \mathcal{S} \to \mathcal{G}$, that maps each state $s$ to the group it belongs.

Now let's introduce a vector $\vec{\theta_t} \in \mathbb{R}^{\lvert \mathcal{G} \rvert}$, which represents the set of all group values at any time step $t$. In other words, $\vec{\theta_t} = \Big( \theta_t(1), \theta_t(2), ..., \theta_t(\lvert \mathcal{G} \rvert) \Big) \doteq \Big( V_t(grp_1), V_t(grp_2), ..., V_t(grp_{\lvert \mathcal{G} \rvert}) \Big)$, where we employed the usual value function $V$ applied to a group, not to a state. But since the state aggregation method implies that the value of a state is always linked to the value of its group, by a slight abuse of notation, we can also consider true that $V(s) = V(g(s))$ for all $s$.

With this notation in place, we can now write the update rule for this method, which applies to groups, not states. Thus at any time step $t$, for any state $s$, its group's value $V(g(s))$ is updated as:

$$
\begin{aligned}
    V_{t+1}(g(s)) & \leftarrow \begin{cases} 
        V_{t}(g(s)) + \alpha \Big[ R_t - V_t(g(s)) \Big] && \text {if } s = s_t \\
        V_{t}(g(s)) && \text {if } s \ne s_t \\
    \end{cases} \\ \\
    & = V_{t}(g(s)) + \alpha \Big[ R_t - V_t(g(s_t)) \Big] \cdot \mathcal{I}_{ss_t} \text {, where } \mathcal{I}_{ss_t} \text { is the usual indicator}. \\
    \\
    \text {Which by definition is equivalent to:} \\
    \theta_{t+1}(g(s)) & = \theta_{t}(g(s)) + \alpha \Big[ R_t - V_t(g(s_t)) \Big] \cdot \mathcal{I}_{ss_t} \text {, for all } s \\
\end{aligned}
$$

Since we defined the vector $\vec{\theta_t}$ to contain all group values $V_t(\cdot)$, we can now write a vectorized version of this update, such that the values of all groups are updated at once. Let's define $\vec{\mathcal{I}}_{gs_t} = \Big( 0, 0, ..., 1, ..., 0 \Big)$, a one-hot vector of size $\lvert \mathcal{G} \rvert$, that indicates to which group belongs $s_t$. The vectorized update rule then becomes:

$$
\begin{aligned}
    \vec{\theta}_{t+1} & = \vec{\theta_{t}} + \alpha \Big[ R_t - V_t(g(s_t)) \Big] \cdot \vec{\mathcal{I}}_{gs_t} \\
    & = \vec{\theta_{t}} + \alpha \Big[ R_t - V_t(g(s_t)) \Big] \cdot \nabla_{\vec{\theta_t}} V_t(g(s_t)) \qquad (1) \\
    & = \vec{\theta_{t}} + \alpha \Big[ R_t - V_t(s_t) \Big] \cdot \nabla_{\vec{\theta_t}} V_t(s_t) \text {, which is exactly (8.3)}. \\
\end{aligned}
$$

Notice the equality in (1) is true because by definition $V_t(g(s_t))$ depends only on a single component of the vector $\vec{\theta}$, namely $\theta(g(s_t))$, so its gradient w.r.t $\vec{\theta_t}$ is simply the one-hot vector: $\Big( \dfrac {\partial \theta(g(s_t))} {\partial \theta(1)}, \dfrac {\partial \theta(g(s_t))} {\partial \theta(2)}, ..., \dfrac {\partial \theta(g(s_t))} {\partial \theta(g(s_t))}, ..., \dfrac {\partial \theta(g(s_t))} {\partial \theta(\lvert \mathcal{G} \rvert)} \Big) = \Big( 0, 0, ..., 1, ..., 0 \Big) = \vec{\mathcal{I}}_{gs_t}$.

Also, for the last equality we use the fact that $V_t(s_t) \doteq V_t(g(s_t))$. This completes the proof that State aggregation can be regarded as a gradient method.